# Chapter 1 Switching,  Lab 1 Configuring Trunks
Scripting and adds on created by jpotts15, all other rights reserved by original authors 
github: https://github.com/jpotts15/poor_lazy_ccie_journey
gitpages: https://jpotts15.github.io/poor_lazy_ccie_journey/

This script will help accelerate the labs from "CCIE Enterprise Infrastructure Foundations" by Narbik

These scripts assume you are using the original EVE-NG labs from the companion materials from the book, I'm currently running eve-ng community but plan on upgrading to EVE-NG pro here soon. Functionally I'm trying to mainly push via telnet to devices directly which should feasibly make it neutral of EVE/GNS3/CML/etc but I don't have the inclination to build all of it out and test

Since the goal of this (and most people who may run into this) is to study CCIE I'll assume a limited knowledge of python, scripting, juypter, etc. as most networks have a different background. My code is probably at the level of a middle schooler these days but its not really my focus so for anyone better at python feel free to improve on it via github.  I will also purposely keep scripts simple to improve readability even if it comes at the cost of efficency or elegance. 

In this juyper notebook i'm going to spell a lot out directly instead of just loading stuff from files in modules and whatnot, doing this to give folks a sense of whats going on but it does make it very verbose, labs after this i'll probably just have a file i'll load and pull modules from to get to the point which is CCIE lab'ing

I also plan on building a few labs as I've noticed some content that I didn't see covered but more on that to come

Lastly I plan on packaging this up to simply python files to run for those who don't want to use juypter or otherwise just want to run a command and be done

## Prework/ Prechecks

0. There are some scripts that talk to the eveng server directly to maybe make things easier, these are safe to skip if your lab is setup, I mainly put these here as for whatever reason my eveng labs like to boot IOL images as L3 by default so I have to manually change this  by hand and the whole point of automating this is to script this once so its quicker time to actually lab things up

### Skip to here if your lab is running and has IOL L2 images assigned to all the nodes

## Lab work

1. This will load the initial configs from pg 4, also built a self contained telnet function as telnetlib is being depreciated

2. 


People unfamiliar with python or juypter notebooks, don't worry, its fairly easy, press shift enter to run cells and follow along. Plenty of python basics courses explain juypter as well so that would be a good place to checkout in case there are issues. 

## Juypter Notebook Basics
0. Always fill in the cells that ask for variables
1. shift+enter to run the cell
2. If the code being run has an output it will display right under it
3. The blocks are sequencial, make sure to run them in order or else the block might try to reference something above it that wasn't run
4. If something goes wrong it will likely just spit an error out, hopefully will just be a missed variable, if not try and see if you can work through it otherwise just do whatever the step is doing by hand and continue, if not possible then report the issue on the github page
5. If something hangs then you'll see the kernel run for a long time (most stuff should be done in a few minutes and you should see the progress). If the kernel hangs go to the kernel menu on top and restart the kernel. If you have to restart the kernel make sure to rerun all the cells needed


'''

# Section 0: Prep work and checking the lab
## Feel free to skip if you are familiar with the labs and don't need to do anything outlined in the overview

## Note: Running the API against EVE will log you out, any labs running will remain running

I made this block basically because my topologies always start with the L3 image when they need the L2, I'll probably also reuse
some of this code later

Overview:

1. We'll get the lab path on the eveng server so that we can interact with the lab programmatically
2. Get the lab, lab device types and ensure they are L2, if not change them by getting available types and changing it
3. anything else i run into later


In [15]:
## Section 0 ##
# for step 0 we'll need the eve-ng sdk (called eve-ng in pypi) written by ttafsir (https://github.com/ttafsir/evengsdk)
# I could write my own api system but thats too far of a tangent and out of scope for this

# Run the command below if in linux/macos
# pip install eve-ng

# if you are using anaconda (recommended):
# On windows open a anaconda prompt (shortcut is created automatically on install) and type: pip install eve-ng
# On Mac if using anaconda easy way to get terminal in your anaconda python install is to go to anaconda navigator, go to enviorments and click the "play" like button next to the enviorment (i.e. base) and click open terminal, then do pip install eve-ng


# if you run this cell it will try running pip for you but really a swing in the dark if it will work
!pip install eve-ng

The folder you are executing pip from can no longer be found.


In [13]:
## Step 0 ##
# Now we can actually run some tooling to get eveng apis

# Fill in the info below, 
eveng_ip = "192.168.178.35"
username = "admin"
password = "eve"
lab_name = ""
depth = 4

from evengsdk.client import EvengClient


def get_telnet_ports(eveng_ip, username, password, lab):
    telnet_ports = {}
    # Replace this with the actual EVE-NG CLI command to get telnet ports
    client = EvengClient(eveng_ip)
    client.login(username=username, password=password)
    #lab = client.api.get_lab(lab)
    lab_nodes_info = client.api.list_nodes(lab)

    for node in lab_nodes_info['data'].values():
        telnet_ports[node['name']] = node['url'][-4:]

    return telnet_ports

def process_folders(client, folder_info, lab_dict, depth):
    if depth == 0:
        return

    folder_list = client.api.get_folder(folder_info['path'])

    for labs_info in folder_list['data']['labs']:
        lab_dict[labs_info['file']] = labs_info['path']

    for sub_folder_info in folder_list['data']['folders']:
        if sub_folder_info['name'] != '..':
            process_folders(client, sub_folder_info, lab_dict, depth - 1)


# Initialize the EvengClient
client = EvengClient(eveng_ip)
client.login(username=username, password=password)

# Initialize an empty dictionary to store lab information
lab_dict = {}

# Get the list of top-level folders
top_level_folder_list = client.api.list_folders()

# Process labs in top-level folders
for labs_info in top_level_folder_list['data']['labs']:
    lab_dict[labs_info['file']] = labs_info['path']

# Check for subfolders in top-level folders
if len(top_level_folder_list['data']['folders']) > 0:
    # Iterate through each subfolder
    for sub_folder_info in top_level_folder_list['data']['folders']:
        if sub_folder_info['name'] != '..':
            process_folders(client, sub_folder_info, lab_dict, depth=2)  # Set the desired depth

for lab_name, path in lab_dict.items():
    print(f"{lab_name} path: {path}")

test.unl: /test.unl
Narbik Enterprise Architecture.unl: ///Cisco Press/BGP/Narbik Enterprise Architecture.unl
RS-MOCK-LAB.unl: //Cisco Press/RS-MOCK-LAB.unl
Lab 1-Establishing a BGP session using the correct TTL value_1660104148098.unl: ///Cisco Press/BGP/Lab 1-Establishing a BGP session using the correct TTL value_1660104148098.unl
Lab 2-Establishing a Peer session using different methods-T1-4_1660104460500.unl: ///Cisco Press/BGP/Lab 2-Establishing a Peer session using different methods-T1-4_1660104460500.unl
Lab 2-Establishing a Peer session using different methods-T6-7_1660104963408.unl: ///Cisco Press/BGP/Lab 2-Establishing a Peer session using different methods-T6-7_1660104963408.unl
Lab 3-Route Reflector-T1-3_1660105332254.unl: ///Cisco Press/BGP/Lab 3-Route Reflector-T1-3_1660105332254.unl
Lab 3-Route Reflector-T4_1660105567426.unl: ///Cisco Press/BGP/Lab 3-Route Reflector-T4_1660105567426.unl
Lab 4-BGP Backdoor and Conditional Advertisement-T1-6_1660105771380.unl: ///Cisco Pre

In [1]:
# Section 1: Initial setup

In [1]:
# Section 2: Finally getting to the labbing bit of this 

In [1]:
# we'll have to do some up front work to connect to the lab and do some stuff
# the default telnetlib is being depreciated and after trying the called out replacements i just built my own.... with blackjack and hookers
import socket
import selectors
import time

class Telnet:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.sock = socket.create_connection((host, port))
        self.selector = selectors.DefaultSelector()
        self.selector.register(self.sock, selectors.EVENT_READ)

    def send_command(self, command):
        self.sock.sendall(command.encode('utf-8'))

    def receive_response(self):
        data = self.sock.recv(65536)
        return data

    def receive_response_all(self):
        self.sock.setblocking()

    def close(self):
        self.sock.close()

class CiscoTelnet:
    def __init__(self, host, port, username, password):
        self.telnet = Telnet(host, port)
        self.username = username
        self.password = password

    def prompt_identifier(self):
        raw_prompt = self._read_raw_prompt()
        self._send_command('\r\n')
        time.sleep(1)
        raw_prompt = self._read_raw_prompt()
        # grab the new prompt
        prompt = raw_prompt.decode('utf-8')
        return prompt

    def _send_command(self, command):
        self.telnet.send_command(command + '\r\n')

    def _read_raw_prompt(self):
        return self.telnet.receive_response()

    def login(self):
        prompt = self.prompt_identifier()
        retry_counter = 0
        while "#" not in prompt and retry_counter < 5:
            retry_counter += 1
            if "initial" in prompt.lower() or "yes/no" in prompt.lower():
                self._send_command("no")
                time.sleep(30)
                self.prompt_identifier()  # Update prompt after 'no' command

            elif "username" in prompt.lower():
                self._send_command(self.username)
                time.sleep(1)
                self._send_command(self.password)
                self.prompt_identifier()  # Update prompt after login

            elif ">" in prompt:
                self._send_command("en")
                time.sleep(1)
                prompt = self.prompt_identifier()  # Update prompt after 'en' command

    def cisco_send_command(self, command):
        self._send_command(command)
        prompt = self.prompt_identifier()
        time.sleep(1)
        return prompt

    def close(self):
        self.telnet.close()
        
    def blackjack(self):
        return "Read it and weep"
    
    def hookers(self):
        return "You know it baby"

# # Example usage
#     host = "172.24.177.49" # eveng IP
#     port = 32769  # port for device being interacted w/, we'll do a list and go through them
#     username = "" # blank if no console username/ auth is set
#     password = "" # blank if no console password/ auth is set
#     commands = ["show version", "show interfaces"] # example list

#     cisco_napalm(host, username, password, commands, port)


In [3]:
# set the following variables for device ports and your eveng server
devices_dict = {
    "sw1_port": 32777,
    "sw2_port": 32778,
    "sw3_port": 32779,
    "sw4_port": 32780,
    "sw5_port": 32781,
    "sw6_port": 32782}

# run task 1, pg 4 commands
commands = ["en",
            "conf t",
            "interface range e0/0-3,e1/0-3,e2/0-3",
            "shut",
            "vtp domain TST",
            "interface range e0/0-3,e1/0-3,e2/0-3",
            "shut",
            "interface range e3/0-3,e4/0-3,e5/0-3,e6/0-3",
            "shut",
            "vtp domain TST",
            "end",
            "exit"
            ]
# setup the telnet session
eveng_host = "172.24.177.49"
username = "" # this is the username on the switch in the lab, not eveng itself
password = "" # same as username
# run the commands
for device, port in devices_dict.items():
    print(f"Device: {device}, Port {port} commands being send")
    cisco_device = CiscoTelnet(eveng_host, port, username, password)
    cisco_device.login()
    for command in commands:
        output = cisco_device.cisco_send_command(command)
        print(f"Command output for {device[0:3]} command {command}: \n{output}")
    cisco_device.close()

Device: sw1_port, Port 32777 commands being send
Command output for sw1 command en: 
n
Switch#
Switch#
Switch#
Switch#
Switch#
Switch#
Command output for sw1 command conf t: 
onf t
Enter configuration commands, one per line.  End with CNTL/Z.
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
Command output for sw1 command interface range e0/0-3,e1/0-3,e2/0-3: 
nterface range e0/0-3,e1/0-3,e2/0-3
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Command output for sw1 command shut: 
hut
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Command output for sw1 command vtp domain TST: 
tp domain TST
Changing VTP domain name from NULL to TST
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
*Jan 25 15:04

KeyboardInterrupt: 

In [ ]:
# wait for the cell above to finish, its very verbose and to remove verbosity comment out the 
# print(f"Command output for {device[0:3]} command {command}: \n{output}") line
# if anything doesn't send rerun and fail back to manually loading
# telnet module is not the best but seems reliable enough for these purposes, usually would just bootstrap to a oobm like network then ssh reliably but with lab trying to not modify the topology and all that
